In [27]:
# making sure I'm in the right python env
!which python

/Users/surya/Library/Caches/pypoetry/virtualenvs/powr-NZLclV9x-py3.10/bin/python


In [28]:
# importing bunch of libraries all in one go for analysis & mpl fig sizes
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (16, 12)
mpl.rcParams['axes.grid'] = False

## Data Ingestion

- Data Loading 
- Merging

In [29]:
from pathlib import Path

# checking current working directory 
print(Path.cwd())

# setting the path to the data directory
data_dir = Path.cwd() / 'data'
print(data_dir)

# setting path to the raw data directory & looking at the files in it
raw_data_dir = data_dir / 'raw'
print(raw_data_dir)
print(os.listdir(raw_data_dir))

/Users/surya/Development/surya/powr
/Users/surya/Development/surya/powr/data
/Users/surya/Development/surya/powr/data/raw
['2022-May.csv', '2022-October.csv', '2022-March.csv', '2022-September.csv', '2022-July.csv', '2022-August.csv', '2022-April.csv', '2022-June.csv']


In [30]:
## loading individual files into dataframes & inspecting them
abs_fpaths_raw_data_files = [ fpath.absolute() for fpath in raw_data_dir.glob('*.csv') ]
print(abs_fpaths_raw_data_files)

# loading the data into dataframes
df_raw_data_list = [ pd.read_csv(fpath) for fpath in abs_fpaths_raw_data_files ]
print(len(df_raw_data_list))


[PosixPath('/Users/surya/Development/surya/powr/data/raw/2022-May.csv'), PosixPath('/Users/surya/Development/surya/powr/data/raw/2022-October.csv'), PosixPath('/Users/surya/Development/surya/powr/data/raw/2022-March.csv'), PosixPath('/Users/surya/Development/surya/powr/data/raw/2022-September.csv'), PosixPath('/Users/surya/Development/surya/powr/data/raw/2022-July.csv'), PosixPath('/Users/surya/Development/surya/powr/data/raw/2022-August.csv'), PosixPath('/Users/surya/Development/surya/powr/data/raw/2022-April.csv'), PosixPath('/Users/surya/Development/surya/powr/data/raw/2022-June.csv')]
8


In [31]:
# inspecting the structure of the dataframes
[ print(df.info()) for df in df_raw_data_list ]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22809 entries, 0 to 22808
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CREATED_AT  22809 non-null  object 
 1   NAME        22809 non-null  object 
 2   VALUE       22809 non-null  float64
 3   UNIT        22809 non-null  object 
dtypes: float64(1), object(3)
memory usage: 712.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20606 entries, 0 to 20605
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CREATED_AT  20606 non-null  object 
 1   NAME        20606 non-null  object 
 2   VALUE       20606 non-null  float64
 3   UNIT        20606 non-null  object 
dtypes: float64(1), object(3)
memory usage: 644.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25128 entries, 0 to 25127
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------

[None, None, None, None, None, None, None, None]

In [32]:
[ print(df.isna().sum()) for df in df_raw_data_list ]

CREATED_AT    0
NAME          0
VALUE         0
UNIT          0
dtype: int64
CREATED_AT    0
NAME          0
VALUE         0
UNIT          0
dtype: int64
CREATED_AT    0
NAME          0
VALUE         0
UNIT          0
dtype: int64
CREATED_AT    0
NAME          0
VALUE         0
UNIT          0
dtype: int64
CREATED_AT    0
NAME          0
VALUE         0
UNIT          0
dtype: int64
CREATED_AT    0
NAME          0
VALUE         0
UNIT          0
dtype: int64
CREATED_AT    0
NAME          0
VALUE         0
UNIT          0
dtype: int64
CREATED_AT    0
NAME          0
VALUE         0
UNIT          0
dtype: int64


[None, None, None, None, None, None, None, None]

Observations:
- All dfs are index on serial number for rows
- All the files have the same four columns
- Dtypes of those columns are the same across files
- No null values in the data

### Merging
- Checking the following
  - same dtypes across dfs
  - same columns


In [33]:
def are_dfs_equivalent(df_list):
    if len(df_list) == 0:
        return False
    
    for df in df_list:
        if df.dtypes.to_dict() != df_list[0].dtypes.to_dict() or df.columns.to_list() != df_list[0].columns.to_list() or df.index.dtype != df_list[0].index.dtype:
            return False
    
    return True

are_dfs_equivalent(df_raw_data_list)

True

In [34]:

df_raw = pd.concat(df_raw_data_list, axis=0, ignore_index=True)
print(df_raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317636 entries, 0 to 317635
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   CREATED_AT  317636 non-null  object 
 1   NAME        317636 non-null  object 
 2   VALUE       317636 non-null  float64
 3   UNIT        317636 non-null  object 
dtypes: float64(1), object(3)
memory usage: 9.7+ MB
None


## Data Cleaning
- Nulls
- Duplicates
- Values that don't make sense
- Date time conversion

### Nulls

In [35]:
df_raw.isna().sum()

CREATED_AT    0
NAME          0
VALUE         0
UNIT          0
dtype: int64

### Duplicate Rows

In [36]:
# checking for duplicate rows
df_raw.duplicated().value_counts()

True     239974
False     77662
dtype: int64

In [37]:
df_raw.value_counts()

CREATED_AT       NAME           VALUE          UNIT
28/9/2022 18:41  powerConsumed   88.445312     W       80
26/9/2022 4:46   powerConsumed   11428.814450  W       70
23/9/2022 7:21   powerConsumed   103.327393    W       70
24/9/2022 10:06  powerConsumed   112.119057    W       65
27/9/2022 4:41   powerConsumed   362.075928    W       65
                                                       ..
25/7/2022 9:16   powerConsumed  -743.721497    W        1
4/6/2022 20:36   powerConsumed   174.626480    W        1
4/6/2022 20:41   powerConsumed   87.952774     W        1
4/6/2022 20:51   powerConsumed   175.559601    W        1
17/6/2022 12:56  powerConsumed   136.005081    W        1
Length: 77662, dtype: int64

### Duplicate values in Specific Columns

In [38]:
## checking for duplicates in specific columns
for col in df_raw.columns:
    print(col, df_raw[col].nunique())
print(len(df_raw))

CREATED_AT 75820
NAME 1
VALUE 62016
UNIT 1
317636


In [39]:
for column in df_raw.columns:
    print(df_raw[column].value_counts())

28/9/2022 18:41    80
26/9/2022 4:46     70
23/9/2022 7:21     70
24/9/2022 10:06    65
27/9/2022 4:41     65
                   ..
28/5/2022 23:46     1
28/5/2022 16:16     1
28/5/2022 18:01     1
28/5/2022 2:51      1
30/6/2022 21:36     1
Name: CREATED_AT, Length: 75820, dtype: int64
powerConsumed    317636
Name: NAME, dtype: int64
 0.000000       26180
 189.000000       184
 88.000000        179
 76.000000        173
 188.000000       170
                ...  
-82.250214          1
 1318.619873        1
 279.512634         1
 518.817810         1
 162.266449         1
Name: VALUE, Length: 62016, dtype: int64
W    317636
Name: UNIT, dtype: int64


### Things that don't make sense from the domain POV
- Duplicates in date time values
- power values that are negative (as it is gross instantaneous consumption)

duplicates only in date time values 

In [40]:
df_raw.loc[~df_raw.duplicated()].CREATED_AT.duplicated().value_counts()

False    75820
True      1842
Name: CREATED_AT, dtype: int64

Negative Gross Power Consumption

In [41]:
df_raw.VALUE.describe().transpose()

count    317636.000000
mean        746.519885
std        1394.164570
min       -8516.939453
25%          99.173634
50%         213.678558
75%         665.042664
max       13626.940430
Name: VALUE, dtype: float64

In [42]:
df_raw[df_raw['VALUE'] < 0]

,CREATED_AT,NAME,VALUE,UNIT
196474,2/7/2022 23:21,powerConsumed,-164.418396,W
196489,2/7/2022 22:16,powerConsumed,-173.122269,W
196503,2/7/2022 1:31,powerConsumed,-1355.698242,W
196510,2/7/2022 7:36,powerConsumed,-267.304749,W
196530,3/7/2022 18:41,powerConsumed,-178.233154,W
...,...,...,...,...
219731,1/7/2022 19:26,powerConsumed,-199.984695,W
219732,1/7/2022 23:41,powerConsumed,-288.000977,W
219755,1/7/2022 23:51,powerConsumed,-222.651321,W
219756,1/7/2022 19:16,powerConsumed,-185.626511,W


In [43]:
df_raw[df_raw['VALUE'] < 0].duplicated().value_counts()

False    2044
True      245
dtype: int64

### Date time converstion

- Visually `'%d/%m/%Y %H:%M'` looks like the correct format, let's test that

In [44]:
try:
    pd.to_datetime(df_raw['CREATED_AT'], format='%d/%m/%Y %H:%M', errors='raise')
except ValueError as e:
    print("It's not the same date format throught out")
    print(e)

It's not the same date format throught out
time data '2022/7/2 08:56' does not match format '%d/%m/%Y %H:%M' (match)


Looking at what other date time formats are present in the data

In [45]:
is_row_in_dmYHM_format = pd.to_datetime(df_raw['CREATED_AT'], format='%d/%m/%Y %H:%M', errors='coerce').isna()
rows_not_in_dmYHM_format = df_raw.loc[is_row_in_dmYHM_format]
rows_not_in_dmYHM_format.value_counts()

CREATED_AT       NAME           VALUE  UNIT
2022/7/5 16:21   powerConsumed  0.0    W       17
2022/7/7 19:46   powerConsumed  0.0    W       15
2022/7/19 19:01  powerConsumed  0.0    W       15
2022/7/21 09:21  powerConsumed  0.0    W       15
2022/7/28 12:11  powerConsumed  0.0    W       14
                                               ..
2022/7/4 05:26   powerConsumed  0.0    W        1
2022/7/14 00:21  powerConsumed  0.0    W        1
2022/7/14 00:16  powerConsumed  0.0    W        1
2022/7/4 05:41   powerConsumed  0.0    W        1
2022/7/9 23:56   powerConsumed  0.0    W        1
Length: 8847, dtype: int64

In [46]:
pd.to_datetime(rows_not_in_dmYHM_format['CREATED_AT'], format='%Y/%m/%d %H:%M', errors='raise')

219784   2022-07-02 08:56:00
219785   2022-07-02 10:46:00
219786   2022-07-02 20:46:00
219787   2022-07-02 19:52:00
219788   2022-07-02 22:46:00
                 ...        
243104   2022-07-01 00:11:00
243105   2022-07-01 04:01:00
243106   2022-07-01 14:16:00
243107   2022-07-01 20:01:00
243108   2022-07-01 00:21:00
Name: CREATED_AT, Length: 23325, dtype: datetime64[ns]

Looks like only two date time formats in data

### Data Cleaning Summary
- drop null values (just in case, even though there are none)
- convert date time column to datetime dtype
- drop duplicated rows
- drop rows where power consumption is negative
- drop columns that are not needed, that have the same value across all rows
- sort rows by date time

In [47]:
# func to convert datatime_strings with multiple formats into datetime objects
def _str_to_datetime(datetime_string, known_strptimes):
    for strptime in known_strptimes:
        trail = pd.to_datetime(datetime_string, format=strptime, errors='coerce', utc=True)
        if pd.notna(trail):
            return trail
    raise ValueError(f"Could not convert {datetime_string} to datetime, tried {known_strptimes}")

In [51]:
def clean_df(raw_dataframe):
    df = raw_dataframe.copy(deep=True)
    df.dropna(how='any', inplace=True)
    # pretty slow but okay for a first pass, can vectorise/optimise later
    df['CREATED_AT'] = df['CREATED_AT'].apply(lambda x: _str_to_datetime(x, ['%d/%m/%Y %H:%M', '%Y/%m/%d %H:%M']))
    df.drop_duplicates(keep="first", ignore_index=True, inplace=True)
    df.drop(df[df['VALUE'] < 0].index, inplace=True)

    nunique = df.nunique()
    cols_to_drop = nunique[nunique == 1].index
    df.drop(cols_to_drop, axis=1, inplace=True)

    df.sort_values(by=['CREATED_AT'], inplace=True, ignore_index=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [52]:
df_clean = clean_df(df_raw)


In [53]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75478 entries, 0 to 75477
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   CREATED_AT  75478 non-null  datetime64[ns, UTC]
 1   VALUE       75478 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 1.2 MB


In [54]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317636 entries, 0 to 317635
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   CREATED_AT  317636 non-null  object 
 1   NAME        317636 non-null  object 
 2   VALUE       317636 non-null  float64
 3   UNIT        317636 non-null  object 
dtypes: float64(1), object(3)
memory usage: 9.7+ MB
